In [1]:
import sys 
sys.path.append('../scripts/')
from dynamic_programming import *

In [2]:
class BeliefDynamicProgramming(DynamicProgramming):  ###beliefdp5（1-3, 21行目）
    def __init__(self, widths, goal, puddles, time_interval, sampling_num, camera, puddle_coef=100.0, \
                 lowerleft=np.array([-4, -4]).T, upperright=np.array([4, 4]).T, dev_borders=[0.1,0.2,0.4,0.8]): #2行目にcamera追加
        super().__init__(widths, goal, puddles, time_interval, sampling_num, puddle_coef, lowerleft, upperright)
        
        self.index_nums = np.array([*self.index_nums, len(dev_borders) + 1])
        nx, ny, nt, nh = self.index_nums
        self.indexes = list(itertools.product(range(nx), range(ny), range(nt), range(nh)))
        
        self.value_function, self.final_state_flags =  self.init_belief_value_function()
        self.policy = np.zeros(np.r_[self.index_nums,2]) 
        
        self.dev_borders = dev_borders
        self.dev_borders_side = [dev_borders[0]/10, *dev_borders, dev_borders[-1]*10]
        self.entropy_state_transition_probs = self.init_entropy_transition_probs()
        self.obs_state_transitions = self.init_observation_probs(camera) #以後observation_updateまで追加
        
    def init_observation_probs(self, camera): ###beliefdp5initobs（下のメソッドも）
        probs = {}
        nx, ny, nt, _ = self.index_nums
        pos_indexes = list(itertools.product(range(nx), range(ny), range(nt)))
        
        for index in pos_indexes: 
            pose = self.pose_min + self.widths*(np.array(index).T + 0.5) #セルの中心の座標
            for i in range(len(self.dev_borders_side)-1):
                sigma = (self.dev_borders_side[i] + self.dev_borders_side[i+1])/2 #範囲の真ん中の標準偏差を遷移前の状態として使う
                S = np.diag([sigma**2, sigma**2, sigma**2]) #sigmaから計算される姿勢の共分散行列
            
                for d in camera.data(pose): #センサ値を繰り返しSに適用
                    S = self.observation_update(d, S, camera, pose)
            
                h = 1.5*np.log(2*np.pi) + 0.5*np.log(np.linalg.det(S)) + 1.5
                sigma_after = np.power(np.e, h/3 - 0.5*np.log(2*np.pi) - 0.5) 
                index_after = self.sigma_to_index(sigma_after)
                probs[(index, i)] = {index_after:1.0}

        return probs
    
    def observation_update(self, d, S, camera, pose):
        distance_dev_rate, direction_dev = 0.14, 0.05 #kf.ipynbから持ってくる
        z = d[0]
        obs_id = d[1]
        
        ###Hの計算###
        mx, my = camera.map.landmarks[obs_id].pos
        mux, muy, mut = pose
        q = (mux - mx)**2 + (muy - my)**2
        sqrtq = np.sqrt(q)
        H = np.array([[(mux - mx)/sqrtq, (muy - my)/sqrtq, 0.0],  [(my - muy)/q, (mux - mx)/q, -1.0]])

        ###Qの計算###
        hmu = IdealCamera.observation_function(pose, camera.map.landmarks[obs_id].pos)
        distance_dev = distance_dev_rate*hmu[0]
        Q = np.diag(np.array([distance_dev**2, direction_dev**2]))
        K = S.dot(H.T).dot(np.linalg.inv(Q + H.dot(S).dot(H.T)))
        return (np.eye(3) - K.dot(H)).dot(S)
        
    def init_entropy_transition_probs(self):
        tmp = {}
        for a in self.actions:
            for i in range(len(self.dev_borders)+1):
                probs = self.entropy_transition_probs(100, self.dev_borders_side[i], self.dev_borders_side[i+1], a)
                tmp[(i, a)] = probs
                
        return tmp
            
    def sigma_to_index(self, sigma):
        for i, e in enumerate(self.dev_borders):
            if sigma < e: return i
            
        return len(self.dev_borders)
        
    def entropy_transition_probs(self, sampling_num, min_sigma, max_sigma, action):
        nu, omega = action
        if abs(omega) < 1e-5: omega = 1e-5
            
        F = np.diag([1.0, 1.0, 1.0])
        F[0, 2] = nu / omega * (math.cos(omega * self.time_interval) - 1.0)#ロボットの向きは関係ないので0[deg]で固定で
        F[1, 2] = nu / omega * (math.sin(omega * self.time_interval))
        
        v ={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2} #移動の誤差モデル（カルマンフィルタのものをコピペ）
        M = np.diag([v["nn"]**2*abs(nu)/self.time_interval + v["no"]**2*abs(omega)/self.time_interval, 
                         v["on"]**2*abs(nu)/self.time_interval + v["oo"]**2*abs(omega)/self.time_interval])

        stw, ctw = math.sin(omega*self.time_interval), math.cos(omega*self.time_interval)
        A = np.array([[stw/omega,    -nu/(omega**2)*stw + nu/omega*self.time_interval*ctw],
                                     [(-ctw + 1.0)/omega, -nu/(omega**2)*(-ctw + 1.0) + nu/omega*self.time_interval*stw],
                                     [0,                                self.time_interval]] )
        
        ans = {}
        for sigma in np.linspace(min_sigma, max_sigma*0.999, sampling_num):
            mat = sigma*sigma*F.dot(F.T) + A.dot(M).dot(A.T)
            h = 1.5*np.log(2*np.pi) + 0.5*np.log(np.linalg.det(mat)) + 1.5 #sigmaが小さすぎると行列式が0以下になるので注意
            sigma_after = np.power(np.e, h/3 - 0.5*np.log(2*np.pi) - 0.5)
            index_after = self.sigma_to_index(sigma_after)
            
            if index_after not in ans:
                ans[index_after] = 0

            ans[index_after] += 1
                
        for e in ans:
            ans[e] /= sampling_num

        return ans
    
    def init_belief_value_function(self): 
        v = np.empty(self.index_nums)
        f = np.zeros(self.index_nums) 
        
        for index in self.indexes:
            f[index] = self.belief_final_state(np.array(index).T)
            v[index] = self.goal.value if f[index] else -100.0
                
        return v, f
        
    def belief_final_state(self, index):
        x_min, y_min, _ = self.pose_min + self.widths*index[0:3] 
        x_max, y_max, _ = self.pose_min + self.widths*(index[0:3] + 1) 
        
        corners = [[x_min, y_min, _], [x_min, y_max, _], [x_max, y_min, _], [x_max, y_max, _] ] 
        return all([self.goal.inside(np.array(c).T) for c in corners ]) and index[3] == 0
    
    def action_value(self, action, index, out_penalty=True):
        value = 0.0
        for delta, prob in self.state_transition_probs[(action, index[2])]:
            after, out_reward = self.out_correction(np.array(index[0:3]).T + delta)
        
            reward = - self.time_interval * self.depths[(after[0], after[1])] * self.puddle_coef - self.time_interval + out_reward*out_penalty
            for h_after, h_prob in self.entropy_state_transition_probs[(index[3], action)].items(): #状態遷移を一段深く
                value += (self.value_function[tuple([*after, h_after])] + reward) * prob * h_prob  #インデックスを拡張し、h_probをかける

        return value

In [3]:
puddles = [Puddle((-2, 0), (0, 2), 0.1), Puddle((-0.5, -2), (2.5, 1), 0.1)] ###beliefdp5exec

##地図とカメラを作る##
m = Map()
for ln in [(1,4), (4,1), (-4,-4)]: m.append_landmark(Landmark(*ln))
c = IdealCamera(m)
    
dp = BeliefDynamicProgramming(np.array([0.2, 0.2, math.pi/18]).T, Goal(-3,-3), puddles, 0.1, 10, c) #カメラを加える

In [4]:
dp.obs_state_transitions

{((0, 0, 0), 0): {0: 1.0},
 ((0, 0, 0), 1): {1: 1.0},
 ((0, 0, 0), 2): {2: 1.0},
 ((0, 0, 0), 3): {3: 1.0},
 ((0, 0, 0), 4): {4: 1.0},
 ((0, 0, 1), 0): {0: 1.0},
 ((0, 0, 1), 1): {1: 1.0},
 ((0, 0, 1), 2): {2: 1.0},
 ((0, 0, 1), 3): {3: 1.0},
 ((0, 0, 1), 4): {4: 1.0},
 ((0, 0, 2), 0): {0: 1.0},
 ((0, 0, 2), 1): {1: 1.0},
 ((0, 0, 2), 2): {2: 1.0},
 ((0, 0, 2), 3): {3: 1.0},
 ((0, 0, 2), 4): {4: 1.0},
 ((0, 0, 3), 0): {0: 1.0},
 ((0, 0, 3), 1): {1: 1.0},
 ((0, 0, 3), 2): {2: 1.0},
 ((0, 0, 3), 3): {3: 1.0},
 ((0, 0, 3), 4): {4: 1.0},
 ((0, 0, 4), 0): {0: 1.0},
 ((0, 0, 4), 1): {1: 1.0},
 ((0, 0, 4), 2): {2: 1.0},
 ((0, 0, 4), 3): {3: 1.0},
 ((0, 0, 4), 4): {4: 1.0},
 ((0, 0, 5), 0): {0: 1.0},
 ((0, 0, 5), 1): {1: 1.0},
 ((0, 0, 5), 2): {2: 1.0},
 ((0, 0, 5), 3): {3: 1.0},
 ((0, 0, 5), 4): {4: 1.0},
 ((0, 0, 6), 0): {0: 1.0},
 ((0, 0, 6), 1): {1: 1.0},
 ((0, 0, 6), 2): {2: 1.0},
 ((0, 0, 6), 3): {3: 1.0},
 ((0, 0, 6), 4): {4: 1.0},
 ((0, 0, 7), 0): {0: 1.0},
 ((0, 0, 7), 1): {1: 1.0},
 